# Getting mask coordinates

Imports

In [1]:

import cv2
import csv
import pandas as pd
from pathlib import Path
import os

Function

In [42]:
def get_mask_coord(mask_path, x_min, x_max, y_min, y_max, out):

    # Load the black-and-white image
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    # Get the original image dimensions
    height, width = mask.shape

    # Calculate scaling factors based on the target ranges
    scale_x = (x_max - x_min) / width
    scale_y = (y_max - y_min) / height

    # Apply binary inverse thresholding to select black regions
    _, binary = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY_INV)

    # Find contours in the black regions
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Open a CSV file to write
    with open(out, mode='w', newline='') as file:
        writer = csv.writer(file)
        
        # Write the header row with 'x' and 'y'
        writer.writerow(['x', 'y'])
        
        # Extract, scale, and write the transformed coordinates of the contours
        for contour in contours:
            for point in contour:
                # Original coordinates (mask space)
                x, y = point[0]
                
                # Scale the x and y coordinates to the target range
                x_transformed = x * scale_x + x_min
                # Correct the y-axis (invert the y-coordinate system)
                y_transformed = y_max - (y * scale_y )
                
                # Write the transformed coordinates to the CSV
                writer.writerow([f"{x_transformed:.4f}", f"{y_transformed:.2f}"])

    print(f"Coordinates have been saved to {out}")

    return None

Code

In [57]:
# regional path
regional_path = "F:/cosmx_data/regional"

# load dimentions csv
dims_table = pd.read_csv(Path(regional_path)/"image_dimensions.csv")
dims_table = dims_table.set_index("sample")



In [58]:
print(dims_table)

                 x_min      x_max     y_min      y_max
sample                                                
s1055ant_bs1  0.111982   4.221500  6.768276  12.183444
s1055ant_bs2  6.381753  11.335643  6.719098  12.672764
s1055ant_bs3  0.237725   4.225399 -0.363008   5.370424
s1055ant_bs4  5.672239   9.290289 -0.444247   5.122115
s1055pos_bs1  0.001804   4.603873  6.876537  13.011707
s1055pos_bs2  5.742103  11.366199  6.886876  13.180697
s1055pos_bs3  0.463518   5.517122 -0.316841   5.188659
s1055pos_bs4  6.642443  10.733198 -0.303108   5.796339
s1056ant_bs1  0.201711   4.568511  5.677670  11.267486
s1056ant_bs2  5.967613   9.773422  6.895647  12.385510
s1056ant_bs3  0.682746   4.525722 -0.253793   4.446666
s1056ant_bs4  6.587638  10.688978  0.741730   6.155816
s1056pos_bs1  1.419251   5.893461  6.221256  12.222313
s1056pos_bs2  6.670134  11.274248  5.635001  11.627157
s1056pos_bs3  0.001564   4.292586  0.139195   5.633027
s1056pos_bs4  5.564742  10.647093 -0.377442   5.114827


In [ ]:
# samples list
samples = dims_table["sample"].tolist()

In [106]:
# mask directories dictionary

mask_dirs = {}

for sample in samples:
    # List files in the directory
    sample_masks = Path(regional_path) / "spatial_segmentation_masks" / sample
    files = os.listdir(sample_masks)
    
    # Create a list to hold filtered files
    filtered_files = []
    
    # Loop through the files and add those that don't start with "._"
    for file in files:
        if not file.startswith("._"):
            filtered_files.append(file)
    
    # Assign the filtered files to the dictionary
    mask_dirs[sample] = filtered_files

    break

print(mask_dirs)


{'s1055ant_bs1': ['s1055ant_bs1-cortex.png']}


In [109]:
for sample, mask_list in mask_dirs.items():
    for mask in mask_list:
        print(mask)

        mask_path = Path(regional_path) / "spatial_segmentation_masks" / sample/ mask

        print(mask_path)

        out_dir = f"{regional_path}/segmentation_mask_coords/{sample}"
        os.makedirs(out_dir, exist_ok=True)

        print(out_dir)
        
        # remove png from the mask name
        mask_noext = os.path.splitext(mask)[0]
        print(mask_noext)
        
        get_mask_coord(

            mask_path=mask_path,
            x_min=dims_table.loc[sample]["x_min"],
            x_max=dims_table.loc[sample]["x_max"],
            y_min=dims_table.loc[sample]["y_min"],
            y_max=dims_table.loc[sample]["y_max"],
            out=f"{out_dir}/{mask_noext}_coords.csv"

        )

        break

s1055ant_bs1-cortex.png
F:\cosmx_data\regional\spatial_segmentation_masks\s1055ant_bs1\s1055ant_bs1-cortex.png
F:/cosmx_data/regional/segmentation_mask_coords/s1055ant_bs1
s1055ant_bs1-cortex
Coordinates have been saved to F:/cosmx_data/regional/segmentation_mask_coords/s1055ant_bs1/s1055ant_bs1-cortex_coords.csv
